<a href="https://colab.research.google.com/github/thomasp05/GIF-7005-Projet/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test with Github

In [ ]:
import time

import torch
from dataset import *
from models import *

In [ ]:
dataset = dcm_dataset('..')
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=2, num_workers=4)

In [ ]:
cnn = Vanilla_CNN(width=16).to("cuda:0")
regressor = Simple_regressor(16 * 1024 * 1024).to("cuda:0")

In [ ]:
params = list(cnn.parameters()) + list(regressor.parameters())
optim = torch.optim.Adam(params, lr=1e-4)

loss_fn = torch.nn.BCEWithLogitsLoss()
n_epoch = 1

In [ ]:
for epoch in range(n_epoch):
    timer = time.time()

    for img, target in dataloader:

        optim.zero_grad()

        img, target = img.to("cuda:0"), target.to("cuda:0")

        pred1 = cnn(img)
        pred2 = regressor(pred1.flatten(1, -1))

        loss = loss_fn(pred2, target)

        loss.backward()

        optim.step()

    print("Epoch : {}".format(epoch + 1))
    print("Time elapsed : {}".format(time.time() - timer))